### Recommendations Rank & Knowledge Based: 


In [118]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')
del movies['Unnamed: 0']
del reviews['Unnamed: 0']

In [119]:
reviews.head()

,user_id,movie_id,rating,timestamp,date,month_1,month_2,month_3,month_4,month_5,...,month_9,month_10,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
0,1,68646,10,1381620027,2013-10-12 23:20:27,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
1,1,113277,10,1379466669,2013-09-18 01:11:09,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,2,422720,8,1412178746,2014-10-01 15:52:26,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
3,2,454876,8,1394818630,2014-03-14 17:37:10,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,2,790636,7,1389963947,2014-01-17 13:05:47,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [120]:
reviews.shape

(712337, 23)

In [121]:
movies.head()

,movie_id,movie,genre,date,1800's,1900's,2000's,History,News,Horror,...,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
0,8,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
1,10,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,12,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,91,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [133]:
movies[movies['movie']=='Pandorica (2016)']

,movie_id,movie,genre,date,1800's,1900's,2000's,History,News,Horror,...,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
29191,5131914,Pandorica (2016),Sci-Fi,2016,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [122]:
movies.shape

(31245, 35)

#### 1. How To Find The Most Popular Movies

For this notebook, we have a single task.  The task is that no matter the user, we need to provide a list of the recommendations based on simply the most popular items.

For this task, we will consider what is "most popular" based on the following criteria:

* A movie with the highest average rating is considered best
* With ties, movies that have more ratings are better
* A movie must have a minimum of 5 ratings to be considered among the best movies
* If movies are tied in their average rating and number of ratings, the ranking is determined by the movie that is the most recent rating

With these criteria, the goal for this notebook is to take a **user_id** and provide back the **n_top** recommendations.  Use the function below as the scaffolding that will be used for all the future recommendations as well.

In [123]:
def popular_recommendations(movies,user_id, n_top):
    '''
    INPUT:
    user_id - the user_id of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''
    reviews['date']=pd.to_datetime(reviews['date'])
    mask=reviews.groupby(['movie_id']).agg({'movie_id':'count'})
    mask.rename(columns={'movie_id':'ratings_freq'},inplace=True)
    mask=mask.sort_values(by=mask.columns[0],ascending=False)
    mask=mask[mask['ratings_freq']>=5]
    
    mask_2=reviews.groupby(['movie_id']).agg({'rating':'mean'})
    mask_2.rename(columns={'rating':'avg_rating'},inplace=True)
    mask_2=mask_2.sort_values(by=mask_2.columns[0],ascending=False)
    
    movies=movies.merge(mask, on='movie_id', how='inner')
    movies=movies.merge(mask_2, on='movie_id', how='inner')
    movies=movies.merge(reviews[['movie_id','date']],on='movie_id',how='inner')
    
    movies=movies.sort_values(by=['avg_rating','ratings_freq','date_y'],ascending=False)
    movies.drop_duplicates(subset=['movie_id'], keep='first',inplace=True)
    top_movies=movies.head(n_top)


    return top_movies['movie'].to_list() # a list of the n_top movies as recommended

In [124]:
popular_recommendations(movies,1234, 10)

['MSG 2 the Messenger (2015)',
 'Avengers: Age of Ultron Parody (2015)',
 'Sorry to Bother You (2018)',
 'Selam (2013)',
 "Quiet Riot: Well Now You're Here, There's No Way Back (2014)",
 'Crawl Bitch Crawl (2012)',
 'Make Like a Dog (2015)',
 'Pandorica (2016)',
 'Third Contact (2011)',
 'Romeo Juliet (2009)']

After Implementing the above function go to the practical quiz in maharatech link and solve the MCQ there. 

In [ ]:
# Put your solutions for MCQ here

**Notice:** This wasn't the only way we could have determined the "top rated" movies.  You can imagine that in keeping track of trending news or trending social events, you would likely want to create a time window from the current time, and then pull the articles in the most recent time frame.  There are always going to be some subjective decisions to be made.  

If you find that no one is paying any attention to your most popular recommendations, then it might be time to find a new way to recommend, which is what the next parts of the lesson should prepare us to do!


### Part II: Adding Filters

Now that you have created a function to give back the **n_top** movies, let's make it a bit more robust.  Add arguments that will act as filters for the movie **year** and **genre**.  

Use the cells below to adjust your existing function to allow for **year** and **genre** arguments as **lists** of **strings**.  Then your ending results are filtered to only movies within the lists of provided years and genres (as `or` conditions).  If no list is provided, there should be no filter applied.

You can adjust other necessary inputs as necessary to retrieve the final results you are looking for!

The shape of the function should be as below:
```
popular_recs_filtered('1', 20, ranked_movies, years=['2015', '2016', '2017', '2018'], genres=['History', 'Comedy'])
```

In [143]:
def popular_recommendations(movies ,user_id, n_top, years=None, genres=None):
    '''
    INPUT:
    user_id - the user_id of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    years - a list of strings with years of movies
    genres - a list of strings with genres of movies
    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''
    reviews['date']=pd.to_datetime(reviews['date'])
    mask=reviews.groupby(['movie_id']).agg({'movie_id':'count'})
    mask.rename(columns={'movie_id':'ratings_freq'},inplace=True)
    mask=mask.sort_values(by=mask.columns[0],ascending=False)
    mask=mask[mask['ratings_freq']>=5]
    
    mask_2=reviews.groupby(['movie_id']).agg({'rating':'mean'})
    mask_2.rename(columns={'rating':'avg_rating'},inplace=True)
    mask_2=mask_2.sort_values(by=mask_2.columns[0],ascending=False)
    
    movies=movies.merge(mask, on='movie_id', how='inner')
    movies=movies.merge(mask_2, on='movie_id', how='inner')
    movies=movies.merge(reviews[['movie_id','date']],on='movie_id',how='inner')
    
    movies=movies.sort_values(by=['avg_rating','ratings_freq','date_y'],ascending=False)
    movies.drop_duplicates(subset=['movie_id'], keep='first',inplace=True)
    movies[['g1','g2','g3']]=movies['genre'].str.split('|',-1,expand=True)
    
    if (genres==None) & (years==None) :
         pass
    elif (genres==None) & (years !=None):
        movies=movies[top_movies['date_x'].isin(years)]
        top_movies=movies.head(n_top)
    elif (genres!=None) & (years ==None):
         movies=movies[movies['g1'].isin(genres) | movies['g2'].isin(genres)| movies['g3'].isin(genres)]
         top_movies=movies.head(n_top)
           
    else:
        movies=movies[(movies['date_x'].isin(years)) & (movies['g1'].isin(genres) | movies['g2'].isin(genres)| movies['g3'].isin(genres))]
        top_movies=movies.head(n_top)
        
    return top_movies['movie'].to_list() # a list of the n_top movies as recommended

In [144]:
popular_recommendations(movies ,1, 10, [1997], ['Drama','Crime'])

['La vita è bella (1997)',
 'Masumiyet (1997)',
 'Mrs Brown (1997)',
 "Eve's Bayou (1997)",
 'Bacheha-Ye aseman (1997)',
 'Selena (1997)',
 "Knockin' on Heaven's Door (1997)",
 'L.A. Confidential (1997)',
 'Good Will Hunting (1997)',
 'Titanic (1997)']

In [ ]:
# Put your solutions for MCQ here

After Implementing the above function go to the practical quiz in maharatech link and solve the MCQ there. 